In [15]:
#install.packages("remotes")
#remotes::install_github("SMLMS/pguIMP")
install.packages("SMLMS/pguIMP")

Installing package into 'C:/Users/lopez/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

Warning message:
"package 'SMLMS/pguIMP' is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages"


In [20]:
#install.packages("remotes")
#remotes::install_github("SMLMS/pguIMP")
#install.packages("SMLMS/pguIMP")
library(SMLMS/pguIMP)
library(tidyverse)
library(gifski)
library(gganimate)
library(plotly)
library(RColorBrewer)
library(wesanderson)

ERROR: Error in library(SMLMS/pguIMP): 'package' must be of length 1


In [7]:
# codigo limpieza y manipulación 
casos_diarios <-  read.csv(file = "Casos_Diarios_Estado_Nacional.csv")
casos_diario_list <- gather(casos_diarios,key ="dias",value="infectados",4:856)

normalizarfecha <- function(fecha){
    return(str_replace_all(fecha,"X","")  %>% str_replace_all("\\.","/"))
}

cod_Meses <- function(x) {
    labels_meses <- c(
        "Ene","Feb","Mar","Abr","May","Jun",
        "Jul","Ago","Sep","Oct","Nov","Dic")
    return(factor(x,levels = (1:12),labels = labels_meses))
}


cod_dias <- function(x) {
    labels_dias <- c(
        "Miercoles","Jueves","Viernes","Sabado","Domingo","Lunes",
        "Martes")
    return(factor(x,levels = (1:7),labels = labels_dias))
}



fecha = map(casos_diario_list[4],normalizarfecha)
casos_diarios_fecha <- cbind(casos_diario_list[c(TRUE,TRUE,TRUE,FALSE,TRUE)],fecha)
casos_diarios_fecha <- casos_diarios_fecha[c(1,2,3,5,4)]

datos <- separate( data = casos_diarios_fecha,
                   col =dias,
                   into = c("dia","mes","año"),
                   sep = '/',
                   remove = FALSE)


#recordatorio
datos <- datos %>% 
    mutate(mest_txt = cod_Meses(as.integer(mes))) %>% 
    mutate(consecutivo = 1:28149) %>% 
    transform(dias = as.Date(dias,"%d/%m/%Y"))
datosnacional <-  datos %>% 
    filter(nombre == "Nacional") %>% 
    mutate(consecutivo = 1:853)%>% 
    filter(año == "2020")


datosnacional$diasem<-format(as.Date(datosnacional$dias),"%A")
#unique(datos$diasem)
datos2 <- datosnacional

datos2$infectados[datos2$diasem == "sábado"] <- NA 
datos2$infectados[datos2$diasem == "domingo"] <- NA


In [8]:
# ejercicio
datos2 <- select(datos2,cve_ent,poblacion,infectados,consecutivo)

# Fill in NA values with the values of the nearest neighbours
datosimputed<- knnImputation(datos2, k = 5, scale = FALSE, meth = "weighAvg", distData = NULL)

resultado <- rbind(datosimputed, select(datosnacional,cve_ent,poblacion,infectados, consecutivo,nombre)) 
#resultado

ggplot(resultado,aes(x = consecutivo, y = infectados, group = nombre, color = as.factor(nombre)))+
                geom_point()+geom_line()


ERROR: Error in knnImputation(datos2, k = 5, scale = FALSE, meth = "weighAvg", : no se pudo encontrar la función "knnImputation"
